<a href="https://colab.research.google.com/github/kujoki/AutoGluon_Text/blob/main/GLUON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 8.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [2]:
!pip3 install -U setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip3 install "torch>=1.0,<1.12+cpu" -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html


In [4]:
!pip3 install --pre autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.7/272.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━

In [5]:
%matplotlib inline

import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
np.random.seed(123)

Сначала рассмотрим набор данных Stanford Sentiment Treebank (SST), который состоит из рецензий на фильмы и связанных с ними настроений. Задача состоит в том, чтобы, получив новую рецензию на фильм, предсказать настроение, отраженное в тексте (в данном случае это бинарная классификация, при которой рецензии помечаются как 1, если они выражают положительное мнение, и как 0 в противном случае). Давайте сначала загрузим и посмотрим на данные, отметив, что метки хранятся в столбце под названием label.

In [6]:
from autogluon.core.utils.loaders import load_pd
train_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/train.parquet')
test_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/dev.parquet')
subsample_size = 1000  # subsample data for faster demo, try setting this to larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head(10)


,sentence,label
43787,very pleasing at its best moments,1
16159,", american chai is enough to make you put away...",0
59015,too much like an infomercial for ram dass 's l...,0
5108,a stirring visual sequence,1
67052,cool visual backmasking,1
35938,hard ground,0
49879,"the striking , quietly vulnerable personality ...",1
51591,pan nalin 's exposition is beautiful and myste...,1
56780,wonderfully loopy,1
28518,"most beautiful , evocative",1


Выше данные хранятся в формате таблицы Parquet, но также можно напрямую загрузить() данные из файла CSV. Хотя здесь загружены файлы из облачного хранилища AWS S3, также могут быть загружены локальные файлы с компьютера. После загрузки train_data представляет собой просто Pandas DataFrame, где каждая строка представляет собой отдельный обучающий пример (чтобы машинное обучение было адекватным, строки должны быть независимыми и одинаково распределенными).

Training

Для того чтобы алгоритм работало быстро, fit() вызывается с подмножеством из 1000 обучающих примеров и его время работы ограничено примерно 1 минутой. Для достижения приемлемой производительности рекомендуется задавать гораздо большее время_лимита (например, 1 час) или вообще не указывать время_лимита (time_limit=None).

In [7]:
from autogluon.text import TextPredictor

predictor = TextPredictor(label='label', eval_metric='acc', path='./ag_sst')
predictor.fit(train_data, time_limit=60)

Global seed set to 123


Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

Training: 0it [00:00, ?it/s]

Time limit reached. Elapsed time is 0:01:00. Signaling Trainer to stop.


Validation: 0it [00:00, ?it/s]

Epoch 0, global step 1: 'val_acc' reached 0.50000 (best 0.50000), saving model to '/content/ag_sst/epoch=0-step=1.ckpt' as top 3
Start to fuse 1 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Выше было указано, что: столбец с именем label содержит значения меток для предсказания, AutoGluon должен оптимизировать свои предсказания для метрики оценки точности, обученные модели должны быть сохранены в папке ag_sst, а обучение должно длиться около 60 секунд.

**Оценка**

После обучения можно легко оценить модель на отдельных тестовых данных, отформатированных аналогично нашим тренировочным данным.

In [8]:
test_score = predictor.evaluate(test_data)
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.5217889908256881}


По умолчанию функция evaluate() выводит метрику оценки, указанную ранее, в примере это точность. Можно указать дополнительные метрики, например, F1 score, при вызове функции evaluate.

In [9]:
test_score = predictor.evaluate(test_data, metrics=['acc', 'f1'])
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.5217889908256881, 'f1': 0.6351706036745406}


**Предсказание**

predictor.predict() - получение предсказания от моделей

In [10]:
sentence1 = "it's a charming and often affecting journey."
sentence2 = "It's slow, very, very, very slow."
predictions = predictor.predict({'sentence': [sentence1, sentence2]})
print('"Sentence":', sentence1, '"Predicted Sentiment":', predictions.iloc[0])
print('"Sentence":', sentence2, '"Predicted Sentiment":', predictions.iloc[1])

Predicting: 0it [00:00, ?it/s]

"Sentence": it's a charming and often affecting journey. "Predicted Sentiment": 1
"Sentence": It's slow, very, very, very slow. "Predicted Sentiment": 0


Для задач классификации можно запросить предсказанные вероятности классов вместо предсказанных классов.

In [11]:
probs = predictor.predict_proba({'sentence': [sentence1, sentence2]})
print('"Sentence":', sentence1, '"Predicted Class-Probabilities":', probs.iloc[0])
print('"Sentence":', sentence2, '"Predicted Class-Probabilities":', probs.iloc[1])

Predicting: 0it [00:00, ?it/s]

"Sentence": it's a charming and often affecting journey. "Predicted Class-Probabilities": 0    0.359767
1    0.640233
Name: 0, dtype: float32
"Sentence": It's slow, very, very, very slow. "Predicted Class-Probabilities": 0    0.687784
1    0.312216
Name: 1, dtype: float32


Прогнозы по всему набору данных

In [12]:
test_predictions = predictor.predict(test_data)
test_predictions.head()

Predicting: 0it [00:00, ?it/s]

0    1
1    0
2    1
3    1
4    0
Name: label, dtype: int64